
##  Text embedding (using Large BERT)

In [1]:
import os
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import torch.functional as F
import pickle
import numpy as np

In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT model and tokenizer
model_name = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# Define your binary classification model
class BERT_Embedder(nn.Module):
    def __init__(self, bert_modele,tokenizer):
        super(BERT_Embedder, self).__init__()
        self.bert = bert_modele
        self.tokenizer = tokenizer
        self.dropout = nn.Dropout(0.1)  
        
        
    def forward(self, input_text):
        inputs = self.tokenizer(input_text, return_tensors="pt").to(device)
        outputs = self.bert(**inputs)
        last_hidden_states = outputs.last_hidden_state[0][0]
        
        # pooled_output = torch.flatten(self.dropout(last_hidden_states))
        
        return last_hidden_states#pooled_output


In [3]:
my_embedder = BERT_Embedder(bert_model,tokenizer).to(device)

Load dataset :


In [180]:
import pandas as pd
df = pd.read_csv('../data/processed/fundemental.csv',index_col=0)
df.head()

,longBusinessSummary,country,industry,sector,forwardPE,beta
OMER,"Omeros Corporation, a clinical-stage biopharma...",United States,Biotechnology,Healthcare,-1.462687,0.957814
CGAU,"Centerra Gold Inc., a gold mining company, eng...",Canada,Gold,Basic Materials,5.580645,1.039580
BFC,Bank First Corporation operates as a holding c...,United States,Banks—Regional,Financial Services,14.056655,0.340248
US.MI,UnipolSai Assicurazioni S.p.A. operates as an ...,Italy,Insurance—Diversified,Financial Services,8.307142,0.637387
001979.SZ,China Merchants Shekou Industrial Zone Holding...,China,Real Estate—Development,Real Estate,11.867925,0.335771


In [19]:
all_texts = df['longBusinessSummary'].to_list()
all_texts[223]

"Dabur India Limited operates as a fast-moving consumer goods company worldwide. It operates through Consumer Care Business, Foods Business, Retail Business, and Other segments. The company manufactures and sells health supplements under the Dabur Chyawanprash, Dabur Vita, Dabur Honey, Dabur Immunity Kit, Dabur Vedic Suraksha Tea, and Dabur Glucose brands; digestive products under the Dabur Hajmola, Pudin Hara, Hingoli, Dabur Nature Care, and Sat Isabgol brands; and shampoos under the Dabur Almond and Vatika brands. It also offers hair oils under the Dabur Amla, Vatika, Dabur Almond, and Anmol brands; skin care products under the Dabur Gulabari, Oxylife, and Dabur Fem brands; and oral care products under the Dabur Red Paste, Dabur Meswak, Dabur Babool, Dabur Lal Dant Manjan, Dabur Red Pulling Oil, and Dabur Dant Rakshak Ayurvedic Paste brands. In addition, the company provides fruit juices under the Real and Real Activ brands; various cooking pastes under the Hommade brand; and milk-ba

In [10]:
X_embedded = []

In [92]:
X_embedded = []  #  223,1164,1252,1380,1491,1767,1832,2159,2245,2462,2832,2963,2969,3499,3587,3951,3994,4480,4679,4805,4810,5406,5433
for index,text in enumerate(all_texts):
    if index>5433:
        X_embedded.append(list(torch.detach(torch.Tensor.cpu(my_embedder(text))).numpy()))
        print(index)

5434
5435
5436
5437
5438
5439
5440


In [93]:
np.shape(X_embedded)

(7, 1024)

In [94]:
np.save('text_embedding23',np.array(X_embedded))

---

In [124]:
dellist = [223,1164,1252,1380,1491,1767,1832,2159,2245,2462,2832,2963,2969,3499,3587,3951,3994,4480,4679,4805,4810,5406,5433]